## 利用word2vec完成下游任务中文文本分类demo

In [6]:
# import pkuseg
import numpy as np
import tqdm
import pandas as pd
import logging
import os
import multiprocessing
import os.path
import gensim
from sklearn.metrics import classification_report
import re
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import jieba


## 对语料库做处理,主要是分词，jieba的好处是快，做demo是好帮手 

In [82]:
## 加载语料库   http://www.sogou.com/labs/sogoudownload/SogouCS/SogouCS.reduced.tar.gz  小型语料库
def build_corpus():
    file_list=os.listdir("data/SogouCS.reduced")
    with open("data/all.txt","w",encoding="utf-8") as fw:
        for file_name in file_list:
            file_name ="data/SogouCS.reduced/" +file_name
            print(file_name)
            with open(file_name,"r",encoding='ANSI') as f:
                for line in f.readlines():
                    a=re.findall(r'^<content>(.*)</content>$',line)  # 正则
                    if a:
                        ans.append(jieba.cut(a[0]))
            for line in ans:
                fw.write(" ".join(line))
                fw.write("\n")
build_corpus()

data/SogouCS.reduced/news.sohunews.010806.txt
data/SogouCS.reduced/news.sohunews.020806.txt
data/SogouCS.reduced/news.sohunews.030806.txt
data/SogouCS.reduced/news.sohunews.040806.txt
data/SogouCS.reduced/news.sohunews.050806.txt
data/SogouCS.reduced/news.sohunews.060806.txt
data/SogouCS.reduced/news.sohunews.070806.txt
data/SogouCS.reduced/news.sohunews.080806.txt
data/SogouCS.reduced/news.sohunews.1010806.txt
data/SogouCS.reduced/news.sohunews.1020806.txt
data/SogouCS.reduced/news.sohunews.1030806.txt
data/SogouCS.reduced/news.sohunews.1040806.txt
data/SogouCS.reduced/news.sohunews.1050806.txt
data/SogouCS.reduced/news.sohunews.1060806.txt
data/SogouCS.reduced/news.sohunews.1070806.txt
data/SogouCS.reduced/news.sohunews.1080806.txt
data/SogouCS.reduced/news.sohunews.110806.txt
data/SogouCS.reduced/news.sohunews.1110806.txt
data/SogouCS.reduced/news.sohunews.1120806.txt
data/SogouCS.reduced/news.sohunews.1130806.txt
data/SogouCS.reduced/news.sohunews.1140806.txt
data/SogouCS.reduced/n

## 使用gensim 的word2vec训练语料库
1 个小时左右

In [84]:
program = os.path.basename(sys.argv[0]) # 读取当前文件的文件名
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s',level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
# inp为输入语料, outp1为输出模型, outp2为vector格式的模型
inp = 'data/all.txt'
out_model = 'train_raw_model.model'
out_vector = 'train_raw_word.vector'
 
# 训练skip-gram模型
model = Word2Vec(LineSentence(inp), size=300, window=5, min_count=5,sg=1,negative=5,iter=5,compute_loss=True,
                workers=multiprocessing.cpu_count() )
# 保存模型
model.save(out_model)
# 保存词向量
model.wv.save_word2vec_format(out_vector, binary=False)


ds/s, in_qsize 12, out_qsize 0
2020-04-06 23:39:59,595: INFO: EPOCH 5 - PROGRESS: at 60.19% examples, 342941 words/s, in_qsize 12, out_qsize 0
2020-04-06 23:40:00,634: INFO: EPOCH 5 - PROGRESS: at 60.42% examples, 342940 words/s, in_qsize 11, out_qsize 0
2020-04-06 23:40:01,639: INFO: EPOCH 5 - PROGRESS: at 60.62% examples, 343083 words/s, in_qsize 12, out_qsize 0
2020-04-06 23:40:02,654: INFO: EPOCH 5 - PROGRESS: at 60.84% examples, 343057 words/s, in_qsize 11, out_qsize 0
2020-04-06 23:40:03,656: INFO: EPOCH 5 - PROGRESS: at 61.05% examples, 343179 words/s, in_qsize 12, out_qsize 0
2020-04-06 23:40:04,657: INFO: EPOCH 5 - PROGRESS: at 61.34% examples, 343312 words/s, in_qsize 11, out_qsize 1
2020-04-06 23:40:05,723: INFO: EPOCH 5 - PROGRESS: at 61.51% examples, 343342 words/s, in_qsize 11, out_qsize 0
2020-04-06 23:40:06,746: INFO: EPOCH 5 - PROGRESS: at 61.79% examples, 343474 words/s, in_qsize 10, out_qsize 1
2020-04-06 23:40:07,760: INFO: EPOCH 5 - PROGRESS: at 62.10% examples, 34

## 评估词向量
语料库不算大,迭代步数不算多，不过看着还行

In [2]:
## 测试词向量好坏
model = Word2Vec.load('train_raw_model.model')

testwords = ['金融','上','股票','跌','经济']
for i in range(5):
    res = model.most_similar(testwords[i])
    print (testwords[i])
    print (res)

#比较两个词语的相似度,越高越好
print('"唐山" 和 "中国" 的相似度:'+ str(model.similarity('唐山','中国')))
print('"中国" 和 "祖国" 的相似度:'+ str(model.similarity('祖国','中国')))
print('"中国" 和 "中国" 的相似度:'+ str(model.similarity('中国','中国')))
#使用一些词语来限定,分为正向和负向的
result = model.most_similar(positive=['中国', '城市'], negative=['学生'])
print('同"中国"与"城市"二词接近,但是与"学生"不接近的词有:')
for item in result:
    print('   "'+item[0]+'"  相似度:'+str(item[1]))

result = model.most_similar(positive=['男人','权利'], negative=['女人'])
print('同"男人"和"权利"接近,但是与"女人"不接近的词有:')
for item in result:
    print('   "'+item[0]+'"  相似度:'+str(item[1]))

result = model.most_similar(positive=['女人','法律'], negative=['男人'])
print('同"女人"和"法律"接近,但是与"男人"不接近的词有:')
for item in result:
    print('   "'+item[0]+'"  相似度:'+str(item[1]))
#从一堆词里面找到不匹配的
print("老师 学生 上课 校长 , 有哪个是不匹配的? word2vec结果说是:"+model.doesnt_match("老师 学生 上课 校长".split()))
print("汽车 火车 单车 相机 , 有哪个是不匹配的? word2vec结果说是:"+model.doesnt_match("汽车 火车 单车 相机".split()))
print("大米 白色 蓝色 绿色 红色 , 有哪个是不匹配的? word2vec结果说是:"+model.doesnt_match("大米 白色 蓝色 绿色 红色 ".split()))
#直接查看某个词的向量
print('中国的特征向量是:')
print(model['中国'])

金融
[('金融市场', 0.6681775450706482), ('金融业', 0.6524161100387573), ('国际金融', 0.6123168468475342), ('金融服务', 0.5865875482559204), ('脱媒', 0.5766823887825012), ('张礼卿', 0.5731323957443237), ('焦瑾璞', 0.5685267448425293), ('盈达', 0.5656598806381226), ('纳逊', 0.5619149804115295), ('李稻', 0.5593039989471436)]
上
[('上能', 0.5605118274688721), ('上将', 0.5335066914558411), ('上时', 0.5098011493682861), ('上开', 0.5076988339424133), ('上成', 0.49496138095855713), ('上要', 0.48989248275756836), ('上均', 0.47195738554000854), ('上以', 0.4668389856815338), ('之上', 0.4653054475784302), ('上面', 0.4557574391365051)]
股票
[('型基金', 0.666684627532959), ('公司股票', 0.6093096733093262), ('中邮系', 0.6081211566925049), ('新股', 0.606896162033081), ('股', 0.6019959449768066), ('型次', 0.5995206832885742), ('债券', 0.5954121351242065), ('股价', 0.5931388139724731), ('期权', 0.587432861328125), ('位约', 0.5874106884002686)]
跌
[('涨', 0.7629534006118774), ('大跌', 0.6598039865493774), ('下跌', 0.6592049598693848), ('跌近', 0.6173271536827087), ('先升后', 0.6132336258888

## 对于我们的文本分类任务,进行word2vec预训练后再训练

In [2]:
model = Word2Vec.load('train_raw_model.model')
train_path="train_raw.txt"
model.train(sentences=LineSentence(train_path), epochs=20,total_examples=8000,total_words=180704,compute_loss=True)
model.save("O2Otrain.w2v")

## 建立分类器
baseline1 ：LR

baseline2 ：catboost

baseline3 ：NNmodel

## LR

In [3]:
import sklearn
from sklearn.linear_model import LogisticRegression

model = Word2Vec.load("O2Otrain.w2v")
## 数据之前就打乱过了
train_path="data/train_split.csv"
valid_path="data/valid_split.csv"
df_train=pd.read_csv("data/train_split.csv")
df_valid=pd.read_csv("data/valid_split.csv")

X_train,y_train=df_train["word"],np.array(df_train["label"])
X_valid,y_valid=df_valid["word"],np.array(df_valid["label"])
print(X_train.shape,y_train.shape)
df_train.head()

(8000,) (8000,)


,label,word
0,1,太 难吃 了 ， 什么 吃 的 都 炸 得 黑黑 的 ， 吃 起来 感觉 像 是 地沟 油炸 的
1,0,很 好吃 ， 服务 太 好 了 ， 就是 等 的 太 久 了
2,0,为什么 不 送 上楼 ？？？？ 是 女 孩子 来 送 也 不 上楼 ？
3,0,喜欢 超级 棒
4,0,味道 好 极了 鸭子 口感 好 极了 服务 好 极了


In [4]:
def sentence2vec(X): 
    X_vec=np.zeros((len(X),300))
    for i,sentence in enumerate(X):
        X_vec[i,:]=( np.mean([model[word] for word in sentence.split() if word in model],axis=0) )
        if np.isnan(X_vec[i,:].mean()):print("乱码数据: ", i)
    return np.array(X_vec)
X_vec_train=sentence2vec(X_train)
X_vec_valid=sentence2vec(X_valid)
X_vec_train.shape,X_vec_valid.shape
## 删除nan数据
X_vec_train=np.delete(X_vec_train,5226,0)
y_train=np.array(np.delete(y_train,5226))
print(X_vec_train.shape,y_train.shape)

乱码数据:  5226
(7999, 300) (7999,)


In [7]:
LR=LogisticRegression(C=3)
LR.fit(X_vec_train,y_train)
y_pred=LR.predict(X_vec_valid)

weight_cab= classification_report(y_valid,y_pred)
print(weight_cab)
sklearn.metrics.f1_score(y_pred,y_valid,average='macro')

precision    recall  f1-score   support

           0       0.96      0.98      0.97      1698
           1       0.88      0.78      0.83       302

    accuracy                           0.95      2000
   macro avg       0.92      0.88      0.90      2000
weighted avg       0.95      0.95      0.95      2000



0.899749373433584

## catboost
主要是gpu使用方便

In [8]:
from  catboost  import  CatBoostClassifier
from sklearn.metrics import classification_report

In [9]:
clf=CatBoostClassifier(iterations=1400
                          ,learning_rate=0.1523
                          ,depth=6
                          ,loss_function='MultiClass'
                          ,silent=True
                          ,class_weights=[30,70]
                          ,task_type="GPU"
                           ,devices='0:1'
                          )

clf.fit(X_vec_train,y_train)
y_pred=clf.predict(X_vec_valid)

weight_cab=classification_report(y_valid,y_pred)
print(weight_cab)
sklearn.metrics.f1_score(y_pred,y_valid,average='macro')

precision    recall  f1-score   support

           0       0.97      0.98      0.97      1698
           1       0.88      0.82      0.85       302

    accuracy                           0.96      2000
   macro avg       0.93      0.90      0.91      2000
weighted avg       0.96      0.96      0.96      2000



0.9115256002686405

## TEXTCNN

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

